## Purpose: Create new MIRI imager reference files for FLT1 ##

First import the things that we need from the pipeline code

In [1]:
import os as os
import numpy as np
import pdb as pdb
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

Import the MIRI coordinates code from https://github.com/STScI-MIRI/coordinates and ensure that it is on the PYTHONPATH.  Also ensure that the output data directory is set:

setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

Import the standalone MIRI imager tools that use the FITS distortion files, and set to FLT-1

In [3]:
import miricoord.imager.mirim_tools as mt
mt.set_toolversion('flt1')

Import the MIRI imager pipeline tools that can work with the CRDS distortion files, and set to FLT-1

In [4]:
import miricoord.imager.mirim_pipetools as mpt
mpt.set_toolversion('flt1')

Import the code that actually creates the CRDS distortion files

In [5]:
import miricoord.imager.makecrds.makecrds_ima_flt1 as cdpcreate

Point to the IDT-delivered CDP file that we're using

In [6]:
cdpfile=mt.get_fitsreffile()

Specify our output filenames for the two asdf files

In [7]:
distfile=os.path.join(data_dir,"crds/temp/jwst_miri_imager_distortion_flt1.asdf")

In [8]:
offfile=os.path.join(data_dir,"crds/temp/jwst_miri_filteroffset_flt1.asdf")

Call the code to actually make the reference files

In [9]:
cdpcreate.make_references(cdpfile,{'DISTORTION': distfile, 'FILTEROFFSET':offfile})

Now point to these reference files and load the transform into the function 'xform'

In [10]:
refs = {"distortion": distfile, "filteroffset": offfile}
refs

{'distortion': '/Users/dlaw/Box/MIRI-COMM/MIRI-COMM-Team/Sandbox/dlaw/miricoord_data_dir/crds/temp/jwst_miri_imager_distortion_flt1.asdf',
 'filteroffset': '/Users/dlaw/Box/MIRI-COMM/MIRI-COMM-Team/Sandbox/dlaw/miricoord_data_dir/crds/temp/jwst_miri_filteroffset_flt1.asdf'}

In [11]:
xform=mpt.xytov2v3model('F770W',refs=refs)

The pipeline uses 0-indexed detector pixels; the MIRI ASDF object assumes this form of input and converts as necessary to the reference frames needed by various IDT transforms.  The reference point of the imager is 692.5, 511.5 in this 0-indexed convention; test that it gives the correct v2,v3 (-453.559, -373.814)

In [12]:
x,y=692.5,511.5

In [13]:
v2,v3=xform(x,y)

In [14]:
print(v2,v3)

-453.37849011784346 -373.8105493033865


And the inverse transform

In [15]:
xform.inverse(v2,v3)

(692.5007019828922, 511.4994907442945)

This location (-453.378, -373.811) is the agreed-upon reference point in FLT-1. 
Now run a full test of the transform against a table of known values using a dedicated testing script.  If no output, it worked.

In [16]:
cdpcreate.test_transform(refs=refs)

As of Apr 12 2017 we can't test the bounding box implementation directly because it is build into the pipeline, not the model class.  Therefore we must test this by actually running the pipeline on MIRI Imager test and reading the transform in from there.  This is done with the mirim_imagertest.ipynb notebook

In [17]:
xform

<CompoundModel(offset_0=-0., offset_1=-0., offset_2=-4., c0_4=-12.7875, c1_4=0.025, c0_5=-12.7875, c1_5=0.025, c0_0_7=0.0783365, c1_0_7=-0.01189775, c2_0_7=-0.00125846, c3_0_7=-0.00000822, c4_0_7=0.00000097, c0_1_7=4.4537997, c0_2_7=-0.00013152, c0_3_7=-0.00020098, c0_4_7=0.0000048, c1_1_7=0.00040611, c1_2_7=0.00000036, c1_3_7=-0.00000072, c2_1_7=-0.00011108, c2_2_7=0.00000474, c3_1_7=-0.00000003, c0_0_8=-19.80635784, c1_0_8=4.41777355, c2_0_8=0.00034838, c3_0_8=-0.00001665, c4_0_8=-0.00000004, c0_1_8=-0.00030697, c0_2_8=0.00009279, c0_3_8=-0.00000336, c0_4_8=-0.00000025, c1_1_8=-0.0027408, c1_2_8=-0.00006825, c1_3_8=0.00000374, c2_1_8=-0.00002375, c2_2_8=0.00000051, c3_1_8=0.0000033, c0_0_10=-373.79, c1_0_10=0.9964467, c0_1_10=0.08422568, c0_0_11=-453.63, c1_0_11=0.08422568, c0_1_11=-0.9964467)>